In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [6]:
# !{sys.executable} -m pip install dscribe

In [1]:
# !conda install -c conda-forge rdkit -y

In [1]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel

In [2]:
from pytorch_geometric2 import *

In [3]:
import pandas as pd
import numpy as np
import os
import os.path as osp
import datetime
import random

# import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

# from torch_geometric.data import Data, DataLoader
# import torch
# import torch.nn.functional as F
# from torch_geometric.datasets import Planetoid
# from torch_geometric.datasets import QM9
# from torch_geometric.datasets import TUDataset
# import torch_geometric.transforms as T
# from torch_geometric.nn import GCNConv, ChebConv  # noqa
# from torch.nn import Sequential, Linear, ReLU, GRU
# import torch_geometric.transforms as T
# from torch_geometric.datasets import QM9
# from torch_geometric.nn import NNConv, Set2Set
# from torch_geometric.data import DataLoader
# from torch_geometric.utils import remove_self_loops
# from torch_geometric.data import Data

from torch.utils.data import DataLoader

In [4]:
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR

In [5]:
file_folder = '../../data/input'
os.listdir(file_folder)

['sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'mulliken_charges.csv',
 'train.csv',
 'test.csv',
 'structures.csv',
 'structures']

In [6]:
graph_dir='../../data/temp/pytorch_geometric2'

In [7]:
# run_convert_to_graph(graph_dir='../../data/temp/pytorch_geometric2', csv_dir=file_folder)

In [7]:
train = pd.read_csv(f'{file_folder}/train.csv')

In [8]:
test = pd.read_csv(f'{file_folder}/test.csv')

In [9]:
test_molecule_names = test.molecule_name.unique().tolist()

In [10]:
train_molecule_names = train.molecule_name.unique().tolist()

In [11]:
batch_size=16
node_dim = 93
edge_dim =  6
num_target = 8
save_folder = None
net_file = None
opt_file = None

# test_dataset = ChampsDataset(molecule_names=molecule_names_test, graph_file=graph_dir , csv='train', mode ='train', split=None, augment=None)
# test_loader = DataLoader(test_dataset, batch_size=1, sampler=RandomSampler(test_dataset), drop_last=True, num_workers=16, pin_memory=True, collate_fn=null_collate)

trial = []

cv = KFold(n_splits= 3, shuffle= True, random_state= 42)
splits = cv.split(train_molecule_names) 
for fold_n, (train_index, valid_index) in enumerate(splits):
    
    train_molecule_names_nkfold = [train_molecule_names[i] for i in train_index]
    valid_molecule_names_nkfold = [train_molecule_names[i] for i in valid_index]
    train_dataset = ChampsDataset(molecule_names=train_molecule_names_nkfold, graph_file=graph_dir , csv='train', mode ='train', split=None, augment=None)
    valid_dataset = ChampsDataset(molecule_names=valid_molecule_names_nkfold, graph_file=graph_dir , csv='train', mode ='train', split=None, augment=None)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=RandomSampler(train_dataset), drop_last=True, num_workers=16, pin_memory=True, collate_fn=null_collate)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, sampler=RandomSampler(valid_dataset), drop_last=True, num_workers=16, pin_memory=True, collate_fn=null_collate)
    
    net = Net(node_dim=node_dim,edge_dim=edge_dim, num_target=num_target).cuda()
    if type(net_file)!=type(None):
        net.load_state_dict(torch.load('net', map_location=lambda storage, loc: storage))
    schduler = NullScheduler(lr=0.001)
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()),lr=schduler(0))
    if type(opt_file)!=type(None):
        checkpoint  = torch.load('opt')
        optimizer.load_state_dict(checkpoint['optimizer'])
        
    his = do_train(net, schduler, optimizer, train_loader, valid_loader, verbose = 1)
    if type(test_loader) != type(None):
        df_pred = predict(test_loader).sort_values(by=['id']).reset_index(drop=True)
    
    if type(save_folder) == type(None):
        torch.save(net.state_dict(),f'{save_folder}/net{fold_n}')
        torch.save({'optimizer': optimizer.state_dict()}, f'{save_folder}/opt{fold_n}')
    
    trial.append({'df_pred':df_pred, 'train_his':his})
    break
    
    

RuntimeError: CUDA out of memory. Tried to allocate 412.00 MiB (GPU 0; 7.44 GiB total capacity; 6.78 GiB already allocated; 143.06 MiB free; 36.57 MiB cached)

In [ ]:
df_pred.to_csv('../../data/submission/submission_gnn.csv', index=False)